In [10]:
filename = "2018-06-20-10-44-11.tdms"
keyence_filename= "2018-06-20-10-44-18-KEYENCE.tdms"

In [11]:
import pickle

pickle_file = "_".join([filename, keyence_filename, "proc_data.p"])

try:
    full_data = pickle.load(open(pickle_file, 'rb'))
    print("Loaded the data from " + pickle_file)
except FileNotFoundError:
    print("File not found, load the data again!")
    
# Pop the info out, it can't be processed...
info = full_data.pop("info")

Loaded the data from 2018-06-20-10-44-11.tdms_2018-06-20-10-44-18-KEYENCE.tdms_proc_data.p


In [12]:
from data_tools import *

#limits = [full_data['position']['time'][5000], full_data['position']['time'][5100]]
limits = [full_data['position']['time'][5000], full_data['position']['time'][6000]]

print("Using limits:")
pprint(limits)

# Truncate the data
data = dict(zip(full_data.keys(), [truncate(limits, full_data[k]) for k in full_data]))

print("Truncated dataset:")
for k in data:
    print("{}: \t{} points".format(k, len(data[k][[*data[k]][0]])))

Using limits:
[datetime.datetime(2018, 6, 20, 9, 49, 9, 641440, tzinfo=<UTC>),
 datetime.datetime(2018, 6, 20, 9, 49, 59, 915079, tzinfo=<UTC>)]
Truncated dataset:
set_point: 	82 points
position: 	1000 points
temperature: 	50 points
keyence: 	1135 points


In [13]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *

init_notebook_mode(connected=True)

import colorlover as cl
colors = cl.scales['7']['qual']['Set1']

plotter = Scatter

plots = [
    plotter(x=data['position']['time'],  y=data['position']['data'],  
            name="Stage Position", line={'color':colors[1]}),
    plotter(x=data['set_point']['time'], y=data['set_point']['data'], 
            name="Set Point", line={'color':colors[0], 'shape':'hv'}),
    
    plotter(x=data['keyence']['time'], y=data['keyence']['data1'], 
            name="Sensor 1", yaxis='y5', mode='lines+markers'),
    plotter(x=data['keyence']['time'], y=data['keyence']['data2'], 
            name="Sensor 2", yaxis='y5', mode='lines+markers')
]

layout = {'width': 800, 'height': 800, 'title': '2018-06-20-10-44-11.tdms + 2018-06-20-10-44-18-KEYENCE.tdms', 'xaxis': {'title': 'Time', 'anchor': 'y2', 'domain': [0, 0.7]}, 'yaxis': {'title': 'Stage Position (mm)', 'domain': [0.5, 1]}, 'yaxis2': {'title': 'Temperature (°C)', 'domain': [0, 0.2]}, 'yaxis3': {'title': 'Temperature diff (°C)', 'overlaying': 'y2', 'side': 'right', 'scaleanchor': 'y2'}, 'yaxis4': {'title': 'Transition time (s)', 'domain': [0.55, 1], 'side': 'right'}, 'showlegend': True, 'legend': {'x': 1.1, 'y': 1, 'orientation': 'v'}, 'yaxis5': {'title': 'Keyence Position (mm)', 'scaleanchor': 'y', 'overlaying': 'y', 'side': 'right'}, 'yaxis6': {'title': 'Difference (mm)', 'overlaying': 'y4', 'side': 'left', 'hoverformat': '.4f', 'domain': [0, 0.45]}, 'xaxis2': {'title': 'Log Prob', 'anchor': 'y6', 'domain': [0.75, 1], 'type': 'log', 'hoverformat': '.4f', 'tickangle': -90}, 'barmode': 'overlay'}

In [14]:
# Find the data rates
rate = dict()

for k in data:
    time_delta = [u - t for t, u in zip(data[k]['time'], data[k]['time'][1:])]
    rate[k] = np.mean(time_delta).total_seconds()

print("Data rates (s):")                  
pprint(rate)


Data rates (s):
{'keyence': 0.04429,
 'position': 0.050273,
 'set_point': 0.615818,
 'temperature': 1.000036}


In [15]:
# Pairs to create a mask of pair[0] - pair[1], and store it in pair[0]
# Probably only makes sense to do this for pair[0] faster than pair[1]???
mask_pairs = [
    ('position', 'set_point'),
    ('keyence',  'set_point'),
]

mask_margin = [datetime.timedelta(seconds=                                    - max([rate['position'], rate['keyence']])),
               datetime.timedelta(seconds=np.mean(data['set_point']['trans']) + max([rate['position'], rate['keyence']]))]

print("Mask margin \t{}s".format(mask_margin))

# Make a progress bar
from ipywidgets import FloatProgress
from IPython.display import display
print("Calculating masks")
p = FloatProgress()
display(p)

generate_mask(mask_pairs, mask_margin, data, progress=p)


Mask margin 	[datetime.timedelta(-1, 86399, 949727), datetime.timedelta(0, 0, 168642)]s
Calculating masks


FloatProgress(value=0.0)

Calculating masks
Index found at:  position	0	set_point	0
Index found at:  position	1	set_point	0
Index found at:  position	2	set_point	0
Index found at:  position	3	set_point	0
Index found at:  position	4	set_point	1
Index found at:  position	5	set_point	1
Index found at:  position	6	set_point	1
Index found at:  position	7	set_point	1
Index found at:  position	8	set_point	1
Index found at:  position	9	set_point	1
Index found at:  position	10	set_point	1
Index found at:  position	11	set_point	1
Index found at:  position	12	set_point	1
Index found at:  position	13	set_point	1
Index found at:  position	14	set_point	1
Index found at:  position	15	set_point	1
Index found at:  position	16	set_point	2
Index found at:  position	17	set_point	2
Index found at:  position	18	set_point	2
Index found at:  position	19	set_point	2
Index found at:  position	20	set_point	2
Index found at:  position	21	set_point	2
Index found at:  position	22	set_point	2
Index found at:  position	23	set_point	2
Index fo

Index found at:  position	602	set_point	49
Index found at:  position	603	set_point	50
Index found at:  position	604	set_point	50
Index found at:  position	605	set_point	50
Index found at:  position	606	set_point	50
Index found at:  position	607	set_point	50
Index found at:  position	608	set_point	50
Index found at:  position	609	set_point	50
Index found at:  position	610	set_point	50
Index found at:  position	611	set_point	50
Index found at:  position	612	set_point	50
Index found at:  position	613	set_point	50
Index found at:  position	614	set_point	50
Index found at:  position	615	set_point	50
Index found at:  position	616	set_point	51
Index found at:  position	617	set_point	51
Index found at:  position	618	set_point	51
Index found at:  position	619	set_point	51
Index found at:  position	620	set_point	51
Index found at:  position	621	set_point	51
Index found at:  position	622	set_point	51
Index found at:  position	623	set_point	51
Index found at:  position	624	set_point	51
Index found

Index found at:  keyence	188	set_point	14
Index found at:  keyence	189	set_point	14
Index found at:  keyence	190	set_point	14
Index found at:  keyence	191	set_point	14
Index found at:  keyence	192	set_point	14
Index found at:  keyence	193	set_point	14
Index found at:  keyence	194	set_point	14
Index found at:  keyence	195	set_point	14
Index found at:  keyence	196	set_point	14
Index found at:  keyence	197	set_point	14
Index found at:  keyence	198	set_point	15
Index found at:  keyence	199	set_point	15
Index found at:  keyence	200	set_point	15
Index found at:  keyence	201	set_point	15
Index found at:  keyence	202	set_point	15
Index found at:  keyence	203	set_point	15
Index found at:  keyence	204	set_point	15
Index found at:  keyence	205	set_point	15
Index found at:  keyence	206	set_point	15
Index found at:  keyence	207	set_point	15
Index found at:  keyence	208	set_point	15
Index found at:  keyence	209	set_point	15
Index found at:  keyence	210	set_point	15
Index found at:  keyence	211	set_p

Index found at:  keyence	687	set_point	50
Index found at:  keyence	688	set_point	50
Index found at:  keyence	689	set_point	50
Index found at:  keyence	690	set_point	50
Index found at:  keyence	691	set_point	50
Index found at:  keyence	692	set_point	50
Index found at:  keyence	693	set_point	50
Index found at:  keyence	694	set_point	50
Index found at:  keyence	695	set_point	50
Index found at:  keyence	696	set_point	50
Index found at:  keyence	697	set_point	51
Index found at:  keyence	698	set_point	51
Index found at:  keyence	699	set_point	51
Index found at:  keyence	700	set_point	51
Index found at:  keyence	701	set_point	51
Index found at:  keyence	702	set_point	51
Index found at:  keyence	703	set_point	51
Index found at:  keyence	704	set_point	51
Index found at:  keyence	705	set_point	51
Index found at:  keyence	706	set_point	51
Index found at:  keyence	707	set_point	51
Index found at:  keyence	708	set_point	51
Index found at:  keyence	709	set_point	51
Index found at:  keyence	710	set_p

In [16]:
new_plots = plots[:2] + plots[8:9]
hoverformat = '.4f'
mode="lines"#+markers"

new_plots.append(plotter(x=data['position']['time'], 
                     y=np.ma.MaskedArray(data['position']['data_set_point_data'], 
                                         mask=data['position']['mask_set_point']), 
                     name="Pos - Set point", legendgroup='data_set_point_data',
                     yaxis='y6', mode=mode,
                     line=dict(color=colors[1])
            ))
new_plots.append(Histogram( 
                     y=np.ma.MaskedArray(data['position']['data_set_point_data'], 
                                         mask=data['position']['mask_set_point']), 
                     name="Pos - Set point", legendgroup='data_set_point_data',
                     yaxis='y6', xaxis='x2',
                     marker=dict(color=colors[1]),
                     opacity = 0.4
            ))


new_plots.append(plotter(x=data['keyence']['time'], 
                     y=np.ma.MaskedArray(data['keyence']['data1_position_data'], 
                                         mask=data['keyence']['mask_set_point']), 
                     name="Sensor 1 - Position", legendgroup='data1_position_data',
                     yaxis='y6', mode=mode,
                     line=dict(color=colors[3])
            ))
new_plots.append(Histogram( 
                     y=np.ma.MaskedArray(data['keyence']['data1_position_data'], 
                                         mask=data['keyence']['mask_set_point']), 
                     name="Sensor 1 - Position", legendgroup='data1_position_data',
                     yaxis='y6', xaxis='x2',
                     marker=dict(color=colors[3]),
                     opacity = 0.4
            ))

#new_plots.append(plotter(x=data['keyence']['time'], 
#                     y=np.ma.MaskedArray(data['keyence']['data2_position_data'], 
#                                         mask=data['keyence']['mask_set_point']), 
#                     name="Sensor 2 - Position", legendgroup='data2_position_data',
#                     yaxis='y6', mode=mode,
#                     line=dict(color=colors[4])
#            ))
#new_plots.append(Histogram( 
#                     y=np.ma.MaskedArray(data['keyence']['data2_position_data'], 
#                                         mask=data['keyence']['mask_set_point']), 
#                     name="Sensor 2 - Position", legendgroup='data2_position_data',
#                     yaxis='y6', xaxis='x2',
#                     marker=dict(color=colors[4]),
#                     opacity = 0.4
#            ))

layout['yaxis6'] = dict(
        title = "Difference (mm)",
        overlaying='y4',
        side='left',
        hoverformat=hoverformat
    )
layout['yaxis4']['side'] = 'right'
layout['xaxis2'] = dict(
        title = "Log Prob",
        anchor = "y6",
        domain = [0.75, 1],
        type = "log",
        hoverformat=hoverformat,
        tickangle = -90,
    )

layout['xaxis']['domain'] = [0, 0.7]
layout['yaxis6']['domain'] = [0, 0.45]
layout['yaxis4']['domain'] = [0.55, 1]

layout['legend'] = dict(
        x = 1.1,
        y = 1,
        orientation="v"
    )

layout['barmode']='overlay'

fig = Figure(data=new_plots, layout=layout)
iplot(fig, filename="{} + {} - Difference".format(filename, keyence_filename))

In [17]:
def moving_average(a, n=3) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] /n

# Smooth the difference data to get a repeatability trend??

masked = np.ma.MaskedArray(data['position']['data_set_point_data'], 
                        mask=data['position']['mask_set_point'])

smoothed = moving_average(masked, 10)

new_plots = new_plots[:3]

new_plots.append(plotter(x=data['position']['time'], y=smoothed.data,
                     name="Pos - Set point smooth",
                     yaxis='y6', mode=mode,
                     line=dict(color=colors[2])
            ))

fig = Figure(data=new_plots, layout=layout)
iplot(fig, filename="{} + {} - Smoothed".format(filename, keyence_filename))

In [18]:
# Smooth the difference data to get a repeatability trend??

print(len(data['position']['data_set_point_data']))

masked = np.ma.MaskedArray(data['position']['data_set_point_data'], 
                        mask=data['position']['mask_set_point']).compressed()

print(len(masked))

times = np.ma.MaskedArray(data['position']['time'], 
                         mask=data['position']['mask_set_point']).compressed()

means = np.array([None,] * len(data['set_point']['time']))

for i in range(len(data['set_point']['time'])-1):
    start = find_nearest_index(times, data['set_point']['time'][i])
    stop  = find_nearest_index(times, data['set_point']['time'][i+1])
    
    if start != stop:
        means[i] = np.mean(masked[start:stop])
    
    #print(start)
    #print(stop)
    
    #pprint(data['position']['data_set_point_data'][start:stop])
    #pprint(means[i])


new_plots = new_plots[:3]

new_plots.append(plotter(x=times, y=masked,
                     name="Pos - Set point mean",
                     yaxis='y6', mode=mode,
                     line=dict(color=colors[3])
            ))

new_plots.append(plotter(x=data['set_point']['time'], y=means,
                     name="Pos - Set point mean",
                     yaxis='y6', mode=mode,
                     line=dict(color=colors[2], shape='hv')
            ))

fig = Figure(data=new_plots, layout=layout)
iplot(fig, filename="{} + {} - Smoothed".format(filename, keyence_filename))

1000
645


In [19]:
pprint(data['set_point']['time'])
pprint(data['position']['time'])

[datetime.datetime(2018, 6, 20, 9, 49, 9, 503070, tzinfo=<UTC>),
 datetime.datetime(2018, 6, 20, 9, 49, 10, 120675, tzinfo=<UTC>),
 datetime.datetime(2018, 6, 20, 9, 49, 10, 735357, tzinfo=<UTC>),
 datetime.datetime(2018, 6, 20, 9, 49, 11, 354028, tzinfo=<UTC>),
 datetime.datetime(2018, 6, 20, 9, 49, 11, 969511, tzinfo=<UTC>),
 datetime.datetime(2018, 6, 20, 9, 49, 12, 585267, tzinfo=<UTC>),
 datetime.datetime(2018, 6, 20, 9, 49, 13, 200474, tzinfo=<UTC>),
 datetime.datetime(2018, 6, 20, 9, 49, 13, 815735, tzinfo=<UTC>),
 datetime.datetime(2018, 6, 20, 9, 49, 14, 429234, tzinfo=<UTC>),
 datetime.datetime(2018, 6, 20, 9, 49, 15, 45907, tzinfo=<UTC>),
 datetime.datetime(2018, 6, 20, 9, 49, 15, 660532, tzinfo=<UTC>),
 datetime.datetime(2018, 6, 20, 9, 49, 16, 276164, tzinfo=<UTC>),
 datetime.datetime(2018, 6, 20, 9, 49, 16, 888801, tzinfo=<UTC>),
 datetime.datetime(2018, 6, 20, 9, 49, 17, 505447, tzinfo=<UTC>),
 datetime.datetime(2018, 6, 20, 9, 49, 18, 119256, tzinfo=<UTC>),
 datetime.da